# About
`solid-file-python` is a Python library for creating and managing files and folders in Solid pods. 

# Start using

## Preparation

### Get a Pod
You can get a free Pod either from [solidcommunity.net](https://solidcommunity.net/) or [Inrupt Pod Spaces](https://signup.pod.inrupt.com/).

### Make sure your Pod is publicly accessible.
We assume the Pod we touch in the following examples is publicly accessible. To use `solid-file-python` with private resources, follow instructions in [access private resources](#Advanced-usage).

To make your Pod public accessible, go to your Pod administration console:

#### solidcommunity.net
1. Login admin console: https://[username].solidcommunity.net/
2. Click the **lock** icon on the menu bar.
3. Move and make sure **Everyone** (the globe icon) to/in the **Owners** section (as shown in the screenshot below.)
![](https://user-images.githubusercontent.com/3183314/124853101-e6ea2780-dfd7-11eb-8263-e0d1dfffe719.png)

### Inrupt Pod Spaces 
1. Login admin console: https://podbrowser.inrupt.com/login
2. Click the **FOLDER DETAILS** button on the menu bar.
3. On the sidebar, expand the **Sharing** section.
4. Click **EDIT EDITORS**.
5. Add and make sure **Anyone** is listed in **Editors** section. (as shown in the screenshot below.)
![](https://user-images.githubusercontent.com/3183314/124853945-45fc6c00-dfd9-11eb-9958-16e0f9d98a7f.png)

## Install 
You can obtain solid-file from Python Package Index using the following identifier: `solid-file`, and install it as a usual Python package.

In [ ]:
!pip install solid-file==0.1.1


## Usage example
We currently support basic folder and file CURD operations. Please check all available operations [here](https://github.com/twonote/solid-file-python/blob/master/src/solid/solid_api.py).

### Import and init

In [ ]:
from solid.solid_api import SolidAPI
api = SolidAPI()

Configure for demostration:

In [ ]:
POD_ENDPOINT = 'https://pod.inrupt.com/petertc/' # Pod to demostrate. Replace it with yours.

folder_name = 'testfolder' 
subfolder_name = 'subfolder'
file_name = 'test.md'

Init vars for demostration:

In [ ]:
import uuid
import io

body = '#hello Solid!'
f = io.BytesIO(body.encode('UTF-8'))

def gen_random_str() -> str:
    return uuid.uuid4().hex

base_url = POD_ENDPOINT + 'playground-' + gen_random_str() + '/'
folder_url = base_url + folder_name + '/'
subfolder_url = folder_url + subfolder_name + '/'
file_url = folder_url + file_name

print(f'base_url:{base_url}')
print(f'folder_url:{folder_url}')
print(f'subfolder_url:{subfolder_url}')
print(f'file_url:{file_url}')

base_url:https://pod.inrupt.com/petertc/playground-36c59a28b0dc444fbd8ec03bd844bb03/
folder_url:https://pod.inrupt.com/petertc/playground-36c59a28b0dc444fbd8ec03bd844bb03/testfolder/
subfolder_url:https://pod.inrupt.com/petertc/playground-36c59a28b0dc444fbd8ec03bd844bb03/testfolder/subfolder/
file_url:https://pod.inrupt.com/petertc/playground-36c59a28b0dc444fbd8ec03bd844bb03/testfolder/test.md


### Check item exists

In [ ]:
api.item_exists(folder_url)

False

### Create folder

In [ ]:
api.create_folder(folder_url)
api.item_exists(folder_url)

True

### List items in the folder
`SolidAPI.read_folder()` returns a `FolderData` instance in which you can get information about the folder, includes name, URL, parent folder, and items in the folder. 

In [ ]:
empty_folder = api.read_folder(folder_url)

print(f'Folder name: {empty_folder.name}')
print(f'Folder url: {empty_folder.url}')
print(f'Parent: {empty_folder.parent}')
print(f'Subfolders in the folder: {empty_folder.folders}')
print(f'Files in the folder: {empty_folder.files}')


Folder name: testfolder
Folder url: https://pod.inrupt.com/petertc/playground-36c59a28b0dc444fbd8ec03bd844bb03/testfolder/
Parent: https://pod.inrupt.com/petertc/playground-36c59a28b0dc444fbd8ec03bd844bb03/
Subfolders in the folder: []
Files in the folder: []


### Add a subfolder to the folder
We can see the subfolder while listing the folder after the subfolder is created. 

In [ ]:
api.create_folder(subfolder_url)
folder_data = api.read_folder(folder_url)
print(f'Subfolders in the folder: {list(map(lambda x: x.name, folder_data.folders))}')


Subfolders in the folder: ['subfolder']


### Add a file to the folder
We can see the file while listing the folder after the file is uploaded. 

In [ ]:
api.put_file(file_url, f, 'text/markdown')
folder_data = api.read_folder(folder_url)
print(f'Files in the folder: {list(map(lambda x: x.name, folder_data.files))}')

Files in the folder: ['test.md']


### Get the file from Pod
Download the file we just upload and print its content.

In [ ]:
resp = api.get(file_url)
print(resp.text)

#hello Solid!


### Delete files and folders in the Pod
Items will not exist after deletion.

In [ ]:
from httpx import HTTPStatusError

try:
  api.delete(file_url)
except HTTPStatusError as e:
    if e.response.status_code != 404:
        raise e

try:
  api.delete(subfolder_url)
except HTTPStatusError as e:
    if e.response.status_code != 404:
        raise e

try:
  api.delete(folder_url)
except HTTPStatusError as e:
    if e.response.status_code != 404:
        raise e

print(api.item_exists(file_url))
print(api.item_exists(file_url))
print(api.item_exists(file_url))

False
False
False


# Advanced usage

## Access private resources

Besides accessing public resources as shown in the above sections, you can use `solid-file-python` to access private resources in a Pod as well.

Currently, `solid-file-python` has the capability to access private resources in Pods hosted by [solidcommunity.net](https://solidcommunity.net/) or provisioned by [Node Solid Server](https://github.com/solid/). We are planning to support ESS (Inrupt Pod Spaces) and CSS when related [issues](https://github.com/solid/node-solid-server/issues/1533) are solved.

To access a private resource, firstly, prepare your identity provider, username and password:


In [ ]:
from getpass import getpass

USERNAME = getpass('Plz enter your user name:')
PASSWORD = getpass('Plz enter your password:')
PRIVATE_RES = getpass('Plz enter your private resource for testing, e.g., https://petertc.solidcommunity.net/private/test.md:')
IDP = 'https://solidcommunity.net'

### Read configs from a config file by following code snippet alternatively.

# from google.colab import drive
# drive.mount('/content/drive')

# import json
# configs = json.load(open('/content/drive/MyDrive/solid-file-python-config.json'))   
# POD_ENDPOINT = configs['SOLID_ENDPOINT']
# IDP = configs['SOLID_IDP']
# USERNAME = configs['SOLID_USERNAME']
# PASSWORD = configs['SOLID_PASSWORD']
# PRIVATE_RES = configs['PRIVATE_RES']




Plz enter your user name:··········
Plz enter your password:··········
Plz enter your private resource for testing, e.g., https://petertc.solidcommunity.net/private/test.md:··········


 Provide these information while initializing `SolidAPI`, as shown in the following example:

In [ ]:
from solid.auth import Auth
from solid.solid_api import SolidAPI

auth = Auth()
api = SolidAPI(auth)
auth.login(IDP, USERNAME, PASSWORD)
resp = api.get(PRIVATE_RES)
print(resp.text)

# This is your markdown file

Here be stuff!
